# MIDI parsing and df creation

## Import libraries

In [ ]:
import os
from mido import MidiFile
import mido
import pandas as pd
import numpy as np

## Load MIDI file

In [ ]:
def load_midi():
    
    global midifile_path

    midifile_path = input("Please insert the MIDI file path and type Enter >>>")

    def path_check(path):
        if os.path.exists(path):
            accept_ext = [".mid", ".midi", ".kar"]
            split_path = os.path.splitext(path)
            if split_path[1] not in accept_ext:
                print("Invalid file format. Please insert the path to a standard MIDI file: ")
                return False
            else:
                return True
        else:
            print("Invalid path. Please insert a valid path to a standard MIDI file: ")
            return False

    while not path_check(midifile_path):
        midifile_path = input()

    print("Valid path.")
    
load_midi()

## MIDI file df creation

In [ ]:
# Note names - "s" for sharp - #
notes_names = ["C-1", "Cs-1", "D-1", "Ds-1", "E-1", "F-1", "Fs-1", "G-1", "Gs-1", "A-1", "As-1", "B-1", "C0", "Cs0", "D0", "Ds0", "E0", "F0", "Fs0", "G0", "Gs0", "A0", "As0", "B0", "C1", "Cs1", "D1", "Ds1", "E1", "F1", "Fs1", "G1", "Gs1", "A1", "As1", "B1", "C2", "Cs2", "D2", "Ds2", "E2", "F2", "Fs2", "G2", "Gs2", "A2", "As2", "B2", "C3", "Cs3", "D3", "Ds3", "E3", "F3", "Fs3", "G3", "Gs3", "A3", "As3", "B3", "C4", "Cs4", "D4", "Ds4", "E4", "F4", "Fs4", "G4", "Gs4", "A4", "As4", "B4", "C5", "Cs5", "D5", "Ds5", "E5", "F5", "Fs5", "G5", "Gs5", "A5", "As5", "B5", "C6", "Cs6", "D6", "Ds6", "E6", "F6", "Fs6", "G6", "Gs6", "A6", "As6", "B6", "C7", "Cs7", "D7", "Ds7", "E7", "F7", "Fs7", "G7", "Gs7", "A7", "As7", "B7", "C8", "Cs8", "D3", "Ds8", "E8", "F8", "Fs8", "G8", "Gs8", "A8", "As8", "B8", "C9", "Cs9", "D9", "Ds9", "E9", "F9", "Fs9", "G9"]

def midifile_df_create():
    
    global notes_list_pd_df
    
    ##########################################################################
    #######   MIDI object creation and basic metamessages retrieval   ########
    ##########################################################################

    midifile = MidiFile(midifile_path)  # creates MIDO MidiFile object
    
    print("// File name: " + str(os.path.splitext(os.path.basename(midifile_path))[0]))
    
    num_midi_tracks = len(midifile.tracks)
    print("// Number of track in the MIDI file: " + str(num_midi_tracks))
    
    ticks_per_beat = midifile.ticks_per_beat
    print("// Ticks per beat of the MIDI file: " + str(ticks_per_beat))
    
    for i in range(len(midifile.tracks[0])):
        try:
            midifile.tracks[0][i].tempo
        except:
            pass
        else:
            tempo = midifile.tracks[0][i].tempo
            print("// Tempo of the MIDI file: " + str(tempo))
            bpm = int(round(mido.tempo2bpm(tempo),2))
            print("// BPM of the MIDI file: " + str(bpm))
            break
       
    for i in range(len(midifile.tracks[0])):
        try:
            midifile.tracks[0][i].numerator
        except:
            pass
        else:
            numerator = midifile.tracks[0][i].numerator
            denominator =  midifile.tracks[0][i].denominator
            print("// Time signature: " + str(numerator) + "/" + str(denominator) + "\n")
            break
            

    ##########################################################################
    ####################   Dataframe creation   ##############################
    ##########################################################################
    
    columns_names = ["note_numb", "note_name", "dt_note_on", "t_dt_note_on", "dt_note_off", "t_dt_note_off", "note_lng", "IOI", "prec_rest", "fllw_rest"]
    total_delta_time = 0
    notes_list = []
    note_on_counter = -1
    
    for counter in range(len(midifile.tracks[num_midi_tracks - 1])):
        
        try:  # I check the presence of delta time and possibly sum it to the total_delta_time
            midifile.tracks[num_midi_tracks - 1][counter].time
        except:
            pass
        else:
            total_delta_time += midifile.tracks[num_midi_tracks - 1][counter].time

        try:  # I check if the message is a note
            midifile.tracks[num_midi_tracks - 1][counter].note
        except:
            pass
        else:  # I collect note data

            # if note on

            if (midifile.tracks[num_midi_tracks - 1][counter].type == "note_on" and midifile.tracks[num_midi_tracks - 1][counter].velocity != 0):         
                note_on_counter += 1
                notes_list.append([midifile.tracks[num_midi_tracks - 1][counter].note, notes_names[midifile.tracks[num_midi_tracks - 1][counter].note], midifile.tracks[num_midi_tracks - 1][counter].time, total_delta_time, "tbp", "tbp", "tbp", "tbp", "tbp", "tbp"])  # "tbp" stands for "to be populated"
                if len(notes_list) == 1:
                        notes_list[0][7] = total_delta_time
                if len(notes_list) > 1:
                        notes_list[len(notes_list) - 1][7] = (total_delta_time - notes_list[len(notes_list) - 2][3])                     
                notes_list[note_on_counter][8] = notes_list[note_on_counter][2]        
                if len(notes_list) > 1:
                    notes_list[note_on_counter - 1][9] = notes_list[note_on_counter][8]

            # if note off

            elif midifile.tracks[num_midi_tracks - 1][counter].type == "note_off" or midifile.tracks[num_midi_tracks - 1][counter].velocity == 0:
                for v in range(len(notes_list)):
                    if (notes_list[v][0] == midifile.tracks[num_midi_tracks - 1][counter].note) and (
                            notes_list[v][4] == "tbp"):
                        notes_list[v][4] = midifile.tracks[num_midi_tracks - 1][counter].time
                        notes_list[v][5] = total_delta_time
                        notes_list[v][6] = notes_list[v][5] - notes_list[v][3]
                        break

            notes_list_pd_df = pd.DataFrame(notes_list, columns=columns_names)
            
    notes_list_pd_df.iat[-1, 9] = 0   # manually sets 0 for the pause following the last note
    
    ms_per_tick = tempo/ticks_per_beat/1000
    
    notes_list_pd_df['t_dt_note_on_ms'] = np.rint(notes_list_pd_df['t_dt_note_on'] * ms_per_tick).astype(np.int64)
    notes_list_pd_df['t_dt_note_off_ms'] = np.rint(notes_list_pd_df['t_dt_note_off'] * ms_per_tick).astype(np.int64)
    notes_list_pd_df['note_lng_ms'] = np.rint(notes_list_pd_df['note_lng'] * ms_per_tick).astype(np.int64)
    notes_list_pd_df['IOI_ms'] = np.rint(notes_list_pd_df['IOI'] * ms_per_tick).astype(np.int64)
    notes_list_pd_df['prec_rest_ms'] = np.rint(notes_list_pd_df['prec_rest'] * ms_per_tick).astype(np.int64)
    notes_list_pd_df['bpm'] = ''
    notes_list_pd_df['ticks_per_beat'] = ''
    notes_list_pd_df['numerator'] = ''
    notes_list_pd_df['denominator'] = ''
    notes_list_pd_df.at[0, 'bpm'] = bpm
    notes_list_pd_df.at[0, 'ticks_per_beat'] = ticks_per_beat
    notes_list_pd_df.at[0, 'numerator'] = numerator
    notes_list_pd_df.at[0, 'denominator'] = denominator
    
    for i in range(4):
        cols = list(notes_list_pd_df.columns)
        cols = [cols[-1]] + cols[:-1]
        notes_list_pd_df = notes_list_pd_df[cols]
    
    number_of_notes = len(notes_list_pd_df.index)
    print("// Number of notes: ", str(number_of_notes), "\n")
    
    inst = open('instrument_settings.txt', 'r')   
    instrument_settings = inst.read()
    print(instrument_settings)
    
    notes_list = notes_list_pd_df['note_name'].values.tolist()
    notes_list_string = ",".join(str(element) for element in notes_list)
    
    available_time_list = notes_list_pd_df['IOI_ms'].values.tolist()
    available_time_list[0] = 1000 # The first note has no problems of available time, so a high fixed value is assigned
    available_time_list_string = ",".join(str(element) for element in available_time_list)
    
    print(f"\n//---------------------------melody\n\n\
m = {number_of_notes};\n\n\
note = [{notes_list_string}];\n\n\
availableTime = [{available_time_list_string}];\n\n\
")

    opt = open('optimization_settings.txt', 'r')   
    optimization_settings = opt.read()

    customize_request = input("//Do you want to customize the configuration? Answer Y or N")
    if customize_request == "Y":
        SP_max = input("//Please type the max span vector \(For example 1,1,2\)")
        f_lb = input("//Please type the lower fret of the handy interval \(For example 5\)")
        f_ub = input("//Please type the upper fret of the handy interval \(For example 12\)")
        d_PC = input("//Please type the discomfort converting factor for position change \(For example 12\)")
        d_SC = input("//Please type the discomfort converting factor for string change \(For example 12\)")
        d_HS = input("//Please type the discomfort converting factor for hand spread \(For example 12\)")
        d_DH = input("//Please type the discomfort converting factor for the distance from the handy position \(For example 12\)")
        d_NO = input("//Please type the discomfort converting factor for the use of open strings \(For example 12\)")
        print(f"\n\n//-------------------------configuration\n\
//hand and comfort \n\
SP_max = [{SP_max}]; //maximum spread for finger 2, 3, and 4 (with respet to their base position, i.e., position wrt index finger)\n\
f_lb = {f_lb}; //lower fret of the handy interval\n\
f_ub = {f_ub}; //upper fret of the handy interval\n\
timePerFret = 20; //(constant) time in millisec to move the hand from a fret to the next or the preceeding\n\n\
//discomfort converting factors\n\
d_PC = {d_PC}; //position change\n\
d_SC = {d_SC}; //string change\n\
d_HS = {d_HS};   //hand spread\n\
d_DH = {d_DH};   //distance from the handy position\n\
d_NO = {d_NO};  //number of notes played on open strings\n\n\
//opt preferences [each value >= 0, sum = 1]\n\
alpha_PC = 0.2; //position change\n\
alpha_SC = 0.1; //string change\n\
alpha_HS = 0.2; //hand spread\n\
alpha_DH = 0.1; //distance from the handy position\n\
alpha_NO = 0.4; //number of notes played on open strings")
    else:
        print(optimization_settings)
    
midifile_df_create()


notes_list_pd_df.rename(columns = {'Column1':'Index'}, inplace = True)
notes_list_pd_df.to_csv('out.csv')

# print(notes_list_pd_df.head(5))